In [2]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.feature_extraction.text import CountVectorizer
import pickle as pkl
import networkx as nx
from networkx.readwrite import json_graph
import json

In [11]:
data = pd.read_excel('add_patent.xlsx', sheet_name='분석대상', index_col=None, header=None)
data = data.fillna(' ')
data = data.loc[:, (data != ' ').any(axis=0)]
data = data.iloc[1:,1:-1]
data = data.reset_index(drop=True)
data

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1979,d_5109,H04J,,,,,,,,,,,,,,
1,1980,d_5106,H04L,,,,,,,,,,,,,,
2,1981,d_5107,A01M,,,,,,,,,,,,,,
3,1981,d_5108,C22F,,,,,,,,,,,,,,
4,1983,d_5104,G01R,G01R,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5104,2020,d_0417,G06F,H04L,,,,,,,,,,,,,
5105,2020,d_0418,G01S,B63G,,,,,,,,,,,,,
5106,2020,d_0488,H04N,H04L,H04N,H04N,,,,,,,,,,,
5107,2020,d_0536,G01S,G01S,,,,,,,,,,,,,


In [12]:
whole = data.iloc[:,2:]

In [13]:
whole_merged = whole[3].map(str)
for i in range(3, len(whole.columns)+3):
    whole_merged += ' ' + whole[i].map(str)
whole_merged

0                H04J H04J                            
1                H04L H04L                            
2                A01M A01M                            
3                C22F C22F                            
4             G01R G01R G01R                          
                             ...                      
5104          G06F G06F H04L                          
5105          G01S G01S B63G                          
5106    H04N H04N H04L H04N H04N                      
5107          G01S G01S G01S                          
5108    G01S G01S G01S G02B G02B                      
Name: 3, Length: 5109, dtype: object

In [17]:
whole_idx = whole_merged.index
cv = CountVectorizer()
cv.fit(whole_merged)
results = cv.transform(whole_merged)
whole_ipc = cv.get_feature_names()
ipc_bow = pd.DataFrame(results.toarray(), columns=whole_ipc, index=whole_idx)
ipc_bow

,a01k,a01m,a01n,a41d,a42b,a47b,a61b,a61f,a61h,a61k,...,h04m,h04n,h04q,h04r,h04s,h04w,h05b,h05f,h05h,h05k
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5104,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5105,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5106,0,0,0,0,0,0,0,0,0,0,...,0,4,0,0,0,0,0,0,0,0
5107,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
whole_adj = np.matmul(ipc_bow.values.T, ipc_bow.values)
whole_adj[whole_adj != 0] = 1
np.fill_diagonal(whole_adj, 0)
print(whole_adj)
print(whole_adj.shape)
print(np.count_nonzero(whole_adj))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(330, 330)
3128


In [21]:
whole_ipc = [i.upper() for i in whole_ipc]

In [22]:
whole_adj_idx = pd.DataFrame(whole_adj, columns=whole_ipc, index=whole_ipc); whole_adj_idx

,A01K,A01M,A01N,A41D,A42B,A47B,A61B,A61F,A61H,A61K,...,H04M,H04N,H04Q,H04R,H04S,H04W,H05B,H05F,H05H,H05K
A01K,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
A01M,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A01N,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A41D,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A42B,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H04W,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
H05B,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
H05F,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
H05H,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# with open(graph +'.graph', 'r') as f:
#     data = json.load(f)
# whole_g = json_graph.node_link_graph(obj[0])

In [23]:
whole_G = nx.from_pandas_adjacency(whole_adj_idx)
whole_G_json = json_graph.node_link_data(whole_G)

In [17]:
len(whole_G.nodes())

330

In [18]:
len(whole_G.edges())

1564

In [3]:
with open('pred.graph', 'r') as f:
    data = json.load(f)
pred_g = json_graph.node_link_graph(data)

In [4]:
deg_cen_sorted = dict(sorted(nx.degree_centrality(pred_g).items(), key=lambda item: -item[1]))
deg_cen_sorted_df = pd.DataFrame(deg_cen_sorted.items(), columns=['IPC', 'Deg. Centraility']).iloc[:10]
deg_cen_sorted_df.to_csv('deg_cen.csv', index=False)
deg_cen_sorted_df

,IPC,Deg. Centraility
0,F42B,0.316109
1,G01N,0.267477
2,F41H,0.246201
3,B64D,0.227964
4,G01M,0.215805
5,G01S,0.197568
6,G06F,0.176292
7,B64C,0.173252
8,H01M,0.161094
9,F41G,0.151976


In [5]:
bet_cen_sorted = dict(sorted(nx.betweenness_centrality(pred_g).items(), key=lambda item: -item[1]))
bet_cen_sorted_df = pd.DataFrame(bet_cen_sorted.items(), columns=['IPC', 'Bet. Centraility']).iloc[:10]
bet_cen_sorted_df.to_csv('bet_cen.csv', index=False)
bet_cen_sorted_df

,IPC,Bet. Centraility
0,G01N,0.133550
1,F42B,0.133437
2,F41H,0.095653
3,B64D,0.048937
4,F41F,0.043522
5,H01M,0.043406
6,G01M,0.043017
7,B01J,0.041232
8,G01S,0.038996
9,B64C,0.029897


In [6]:
cl_cen_sorted = dict(sorted(nx.closeness_centrality(pred_g).items(), key=lambda item: -item[1]))
cl_cen_sorted_df = pd.DataFrame(cl_cen_sorted.items(), columns=['IPC', 'Clos. Centraility']).iloc[:10]
cl_cen_sorted_df.to_csv('cl_cen.csv', index=False)
cl_cen_sorted_df

,IPC,Clos. Centraility
0,F42B,0.525060
1,G01N,0.511320
2,F41H,0.498281
3,B64D,0.485029
4,G01M,0.482462
5,G01S,0.468418
6,B64C,0.463655
7,G06F,0.459758
8,F41G,0.452908
9,H01M,0.452908


In [16]:
with open('whole.graph', 'w') as f:
    json.dump(whole_G_json, f)

In [25]:
# whole_adj_idx.to_csv('add_ipc_whole.csv')

#### Test load

In [57]:
with open('original.graph', 'r') as f:
    loaded = json.load(f)

In [58]:
test_load = json_graph.node_link_graph(loaded)

In [59]:
graphs = ['original', 'train', 'test']
obj = []
for graph in graphs:
    with open(graph +'.graph', 'r') as f:
        data = json.load(f)
    obj.append(data)
orig_g = json_graph.node_link_graph(obj[0])
train_g = json_graph.node_link_graph(obj[1])
test_g = json_graph.node_link_graph(obj[2])

In [60]:
print(len(orig_g.nodes()), len(orig_g.edges()))
print(len(train_g.nodes()), len(train_g.edges()))
print(len(test_g.nodes()), len(test_g.edges()))

322 1551
322 1374
153 407
